In [1]:
import boto3
import json
from datetime import datetime
from edc import setup_environment_variables

# Oauth
from oauthlib.oauth2 import BackendApplicationClient
from requests_oauthlib import OAuth2Session

setup_environment_variables()

API credentials have automatically been injected for your active subscriptions.  
The following environment variables are now available:
* `SH_CLIENT_NAME`
* `SH_CLIENT_SECRET`
* `SH_INSTANCE_ID`
* `SH_CLIENT_ID`

The following additional environment variables have been loaded from `~/custom.env`:
* `AWS_BUCKET`
------


In [42]:
# Your client credentials
client_id = %env SH_CLIENT_ID
client_secret = %env SH_CLIENT_SECRET

# Create a session
client = BackendApplicationClient(client_id=client_id)
oauth = OAuth2Session(client=client)

token = oauth.fetch_token(token_url='https://services.sentinel-hub.com/oauth/token',
                          client_id=client_id, client_secret=client_secret)

resp = oauth.get("https://services.sentinel-hub.com/oauth/tokeninfo")

url = 'https://services.sentinel-hub.com/batch/v1/process/'

In [43]:
bucket_name = %env AWS_BUCKET

In [44]:
i = 0

s3 = boto3.resource('s3')
bk = s3.Bucket(bucket_name)

objects = list(bk.objects.all())
request_ids = set([o.key.split('/')[0] for o in objects if '/' in o.key])

for rid in request_ids:
    # Check if measurements.json already exists for a request
    if f'{rid}/measurements.json' in [o.key for o in objects]:
        pass
    else:
        response = oauth.request('GET', f'{url}{rid}')

        if response.ok:
            request = response.json()

            if request['status'] == 'DONE':
                # Collect size
                size = sum([o.size for o in objects if o.key.startswith(rid)])
                
                # Collect execution time
                created = datetime.fromisoformat(request['created'].replace('Z','+00:00'))
                finished = max([o.last_modified for o in objects if o.key.startswith(rid) and
                               o.key.endswith('tif')])
                
                # Collect costs
                tiles = []
                viewtoken = 0
                while viewtoken is not None:
                    response = oauth.request('GET', f'{url}{rid}/tiles?viewtoken={viewtoken}').json()
                    tiles.extend(response['member'])
                    viewtoken=response['view']['nextToken']
                costs = sum([t['cost'] for t in tiles])

                # Save measurements to bucket
                bk.put_object(Key=rid + '/measurements.json', Body=json.dumps({
                    'description': request['description'],
                    'id': rid,
                    'execution time': str(finished-created),
                    'costs': costs,
                    'size': size
                }))
                
                i+=1


print(i, 'measurements written to bucket')

3 measurements written to bucket


In [45]:
import pandas as pd
objects = list(bk.objects.all())
df= pd.DataFrame([json.load(o.get()['Body']) for o in objects if o.key.endswith('measurements.json')])

df[(df.description.str.contains('Test Vorarlberg')) | (df.description == 'Test Mattersburg')]#.plot()

,description,id,execution time,costs,size
2,Test Vorarlberg 16 20180701_20180915,0be2fc59-12cc-4952-9190-0bcdca2ce8cf,0:05:51.438000,1123.458276,290571097
6,Test Vorarlberg 16 20180701_20180915,15207bb6-deed-4646-a2da-d425cfcad1ab,0:05:34.505000,1123.458276,290571097
11,Test Mattersburg,200f77d1-62bb-482a-b764-88d9aacd30ef,0:03:20.798000,306.397705,49554388
17,Test Vorarlberg,2a9c41ee-9a37-4bc8-813e-92b63fcb03d2,0:03:17.785000,889.617226,207190776
31,Test Vorarlberg 16,3f3c7903-4f3d-444f-b6d8-cf8d3fa41ea8,0:08:15.784000,1123.458276,290570855
39,Test Vorarlberg 16 20180701_20180915,4796c0ae-8a12-4626-81f3-1d372c32c84e,0:25:01.153000,6740.749658,1377755527
46,Test Vorarlberg,54042ad0-b451-4ffc-a7bb-2a98c92d6f04,0:03:19.080000,889.617226,207190776
48,Test Vorarlberg 16 20180701_20180915,54c1643c-4ec1-4360-ae3b-ac9f659426a3,0:25:01.084000,6740.749658,1377755527
51,Test Mattersburg,5ddb9c9b-6c95-409e-84f4-aa0112549b5c,0:03:25.216000,306.397705,49554388
56,Test Vorarlberg 16 20180701_20180915,659f3b18-724c-4537-8a5b-8508a0938918,0:05:56.503000,1123.458276,290571097
